In [1]:
import sqlite3
import pandas

In [2]:
pandas.set_option('display.max_rows', 400)

In [3]:
datasets = ["orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]

for data in datasets:
    cnx = sqlite3.connect(f"../../data/{data}.db")
    df = pandas.read_sql_query("""
        SELECT p.trial_id, p.param_name as param, ROUND(p.param_value, 2) as p_value FROM
        trial_values as v JOIN trial_params as p ON v.trial_id = p.trial_id
        WHERE value = ( SELECT MIN(value) FROM trial_values );    """, cnx)
    display(data)
    df_groups = df.groupby(["param", "p_value"]).count()
    display(df_groups)

'orthologs_neuroglobin'

trial_id
param        p_value          
filter_sigma 0.1           141
             0.2            98
filter_size  7.0           239

'orthologs_cytoglobin'

trial_id
param        p_value          
filter_sigma 1.0             9
             1.1            12
             1.2            19
             1.3             6
             1.4             8
             1.5            10
filter_size  3.0            64

'orthologs_androglobin'

trial_id
param        p_value          
filter_sigma 0.3             1
             0.4             1
             0.5             5
             0.6             6
             0.7             5
             0.8             3
             0.9             6
             1.0             3
             1.1             4
             1.3             9
             1.4             1
             1.5             3
filter_size  3.0             7
             4.0            13
             5.0            11
             6.0            11
             7.0             5